In [213]:
%run ./common.ipynb

from dotenv import load_dotenv
load_dotenv()

api_token=os.getenv('JIRA_TOKEN')

if not api_token:
    raise ValueError("No JIRA API token found. Check your .env file / environment variable.")


In [214]:

# run_proj = sys.argv[1]
run_proj ='ALL'


partitionCnt=5
filepath='/Users/u1002018/Library/CloudStorage/OneDrive-SharedLibraries-FootLocker/Global Technology Services - DASH Doc Library/Timesheet/'

# filepath='../output/TimeSheet'
importfile='/Users/u1002018/Library/CloudStorage/OneDrive-SharedLibraries-FootLocker/Global Technology Services - DASH Doc Library/AllProjects/transition_history_Story.csv'

issueLists = [
            # {'issueType':"Portfolio Initiative",'partitionCnt':1},
            # {'issueType':"Product Initiative", 'partitionCnt':1 },
            # {'issueType':"Epic",'partitionCnt':partitionCnt},
            {'issueType':"Story",'partitionCnt':partitionCnt}
            ]
             
pd.set_option('display.max_columns', None)


In [215]:
# Jira ID 
# Name
# Planview Portfolio ID
# Work Description
# IssueType
# Work State
def portoFolioDataframe(dataframe):
    return dataframe.filter(['key', 'fields.summary', 'fields.issuetype.name',
                             'fields.status.name','fields.customfield_32307'
                         #  'fields.description',
        ]).rename(columns={
        'key':'JiraID',
        'fields.summary':'Name',
        'fields.customfield_32307':'Planview Portfolio ID',
        # 'fields.description':'Work Description',
        'fields.issuetype.name':'IssueType',
        'fields.status.name':'Work State'})

In [216]:
# Name
# ParentLink
# Work Description
# Jira ID
# IssueType
# Start date
# End date
# Work State
def productInitDataframe(dataframe):
    return dataframe.filter(['key', 'fields.summary', 'fields.customfield_12823',
                             'fields.issuetype.name','fields.status.name'
                            #  'fields.description',
                            ]).rename(columns={
                            'key':'JiraID',
                            'fields.summary':'Name',
                            'fields.customfield_12823': 'ParentLink',
                            # 'fields.description':'Work Description',
                            'fields.issuetype.name':'IssueType',
                            'fields.status.name':'Work State'})

In [217]:
# Name ✅
# ParentLink ✅
# Work Description ❌
# Jira ID ✅
# IssueType ✅
# Start date ✅
# End date ✅
# Banner ⏳ 
# Region ⏳
# T-shirt Size ⏳
# Epic Work Type ✅
# Work State ✅
# In-Progress DateDate ⏳
# Completed Date ⏳
def EpicDataframe(dataframe):
    return dataframe.filter(['key', 'fields.summary', 'fields.customfield_12823',
                             'fields.issuetype.name','fields.status.name','fields.customfield_12305',
                             'fields.customfield_31400', 'fields.customfield_13008', 'fields.customfield_12101',
                            #  'fields.description',
        ]).rename(columns={
        'fields.summary':'Name',
        'key':'JiraID',
        'fields.customfield_12823': 'ParentLink',
        'fields.customfield_13008': 'Targeted Start Date',
        'fields.customfield_12101': 'Targeted End Date',
        # 'fields.description':'Work Description',
        'fields.customfield_12305':'Banner',
        'fields.customfield_31400': 'Epic Type',
        'fields.issuetype.name':'IssueType',
        'fields.status.name':'Work State'})

In [218]:
# Name ✅
# ParentLink ✅
# Work Description ❌
# Jira ID ✅
# IssueType ✅
# Story Estimate ✅
# Story Work Type ✅
# Work State ✅
# In-Progress Date ✅
# Completed Date ✅
# Jira Project Name ✅
def StoryDataframe(dataframe):
    return dataframe.filter(['key', 'fields.summary', 'fields.customfield_10006',
                             'fields.customfield_10002',
                             'fields.issuetype.name','fields.status.name',
                             'fields.project.name','fields.customfield_31800'
         #  'fields.description',
        ]).rename(columns={
        'fields.summary':'Name',
        'fields.customfield_10006': 'ParentLink', #Epic Link
      # 'fields.description':'Work Description',
        'fields.customfield_31800':'story type',
        'key':'JiraID',
         'fields.issuetype.name':'IssueType',
        'fields.customfield_10002':'StoryPoint',
        'fields.status.name':'Work State',
        'fields.project.name':'projectName'})

In [219]:
global logger
logger = setup_logger()
logger.info("=======================================")
logger.info("Script start accumulating data from JIRA")

In [220]:
def getDates(key,state):
    dates=''
    filterdata = importTransactionFile[importTransactionFile['key']==key]
    if state == 'close':    
     state= filterdata[filterdata['to'].isin(close_state)]
    else:
     state= filterdata[filterdata['to'].isin(development_state)]
    
    if state is not None and not state.empty:
        dates= state['created_at'].values[0]

    return dates

In [221]:
def data_clean(dataframe, issueType):
    if dataframe is not None and not dataframe.empty:
       if issueType == 'Portfolio Initiative':
         dataframe = portoFolioDataframe(dataframe)
       elif issueType == 'Product Initiative':
         dataframe = productInitDataframe(dataframe)
       elif issueType == 'Epic':
          dataframe = EpicDataframe(dataframe)
       elif issueType == 'Story':
          dataframe = StoryDataframe(dataframe)
          dataframe['In-Progress Date'] = dataframe['JiraID'].apply(lambda x: getDates(x,'inProgress'))
          dataframe['Completed Date'] = dataframe['JiraID'].apply(lambda x: getDates(x,'close'))
    return dataframe


In [222]:
def fetch( project, issueType):
    global fetch_df 
    logger.info(f"Fetch {issueType}...")

    # if issueType == 'Story':
    #     attribute =' &expand=projects.issuetypes.fields,changelog'
    # else:
    attribute =' '

    searchQry=f'?jql= project in ({project}) and issueType="{issueType}" and updated > startOfMonth(-11)'

    # if cond_flg == True: 
    #     searchQry=f"{searchQry} and status not in (Cancelled, Done)  {attribute}"
    # else:
    searchQry=f"{searchQry} {attribute}"

    fetch_df = getDataSet(searchQry, project, issueType, api_token )

    return data_clean(fetch_df,issueType )    
    

In [223]:
def getChangeLog(element):
    changelog_Rec = [] 
    for history in element:
        created_at = history.get('created',None)
        for item in history['items']:
            if item['field'] =='status' :
                changelog_Rec.append({
                    'created_at': created_at,
                    'field': item['field'],
                    'from': item['fromString'],
                    'to': item['toString'],
                })
    return changelog_Rec

In [224]:
# Get all .csv files in the 'data' directory
csv_files = Path(filepath).glob("*_working.csv")
for file in csv_files:
    filename=file.name
    os.remove(os.path.join(filepath, filename))

global importTransactionFile 
importTransactionFile = pd.read_csv(importfile)

strings =pd.Series(["PCR"])
# strings = projectList(api_token)

for issueList in issueLists:
    split_string_list = split_list(strings, issueList['partitionCnt'])
    filtered_list = [series for series in split_string_list if not series.empty]
    for partition in filtered_list:
        logger.warning(f"processing %s - %s",partition.values[0],issueList['issueType'])

        df=fetch(', '.join(['"{}"'.format(value) for value in partition]),issueList['issueType'])
        if df is not None and not df.empty:
            try:     
                    output_path = os.path.join(filepath, f"{issueList['issueType']}_working.csv")            
                    df.to_csv(output_path ,mode='a', header=not os.path.exists(output_path), index=False)
            except Exception as e:  
                logger.error(f"Failed to upload files to {filepath}: {e}")
                pass
                
            logger.info("Move working files to current files")
            # Delete the old DataFrame 
            del(df)

            # Perform garbage collection                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
            gc.collect()
            # break
    clean_folder(filepath,"_working.csv")

del(importTransactionFile)
# Perform garbage collection                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
gc.collect()
logger.warning(f"Script ended accumulating data from JIRA ")
logger.warning("=======================================")


2024-12-31 09:11:25,850 - root - WARNING - processing PCR - Story
2024-12-31 09:11:47,997 - root - WARNING - renaming Story_working.csv TO Story.csv....
2024-12-31 09:11:48,099 - root - WARNING - Script ended accumulating data from JIRA 
2024-12-31 09:11:48,100 - root - WARNING - =======================================
